<a href="https://colab.research.google.com/github/m-soro/Variance_Analysis/blob/main/VarianceAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Excel Variance Report Generator
Creates a downloadable summary report of outlet variances in excel format.
1. Export variance report in csv file
2. Drag to Files
3. Click Generate Report
4. Report will auto download

In [ ]:
#@markdown ### Generate Report
try:
    import glob
    import pandas as pd
    import xlsxwriter
    from google.colab import files
except:
    !pip install xlsxwriter

def var_analysis():
    file = glob.glob('*.csv')
    df = pd.read_csv(file[0],encoding='latin-1')
    sheet_name = df['Unnamed: 3'][3]
    date = df['Outlet Variance'][0]
    cut_off_row = df.loc[df['Outlet Variance']=='Account'].index.tolist()
    df.drop(columns=['Outlet Variance','Unnamed: 8','Unnamed: 9'], index =[0,1], inplace=True)
    df.columns = ['Item Num', 'Unit', 'Desc','BeginInv','Purch','Req','Adj','OnHand','Count','Var']
    df.drop([2,3], inplace=True)
    df = df.iloc[:cut_off_row[0]]
    new_var = df['Var'].dropna().tolist()
    new_variance = new_var[1::2]
    df = df.dropna()
    df['Amt'] = new_variance[:-1]
    df.set_index('Item Num')
    writer = pd.ExcelWriter(sheet_name + '.xlsx',
                             engine='xlsxwriter',
                             options={'strings_to_numbers': True})
    df.to_excel(writer,sheet_name=sheet_name,index=False)
    workbook  = writer.book
    worksheet = writer.sheets[sheet_name]
    worksheet.set_column('B:B', df['Unit'].str.len().max())
    worksheet.set_column('C:C', df['Desc'].str.len().max())
    worksheet.set_column('K:K',12)
    formula = str('=SUM(K1:K' + str(len(df)+1)+ ")" )
    bold = workbook.add_format({'bold': True})
    worksheet.write('J'+ str( int(float((len(df))+3)) ) , 'Total Var',bold) 
    print('Downloading Outlet Variance for:' + '\n\n' + sheet_name + '\n')
    print('For variance total: \n\nPaste this on cell ' + 'K'+ str( int(float((len(df))+3)) ) + ' --->  '+ formula)
    worksheet.set_paper(1)
    worksheet.repeat_rows(0) 
    worksheet.set_header("Outlet Variance\n" + sheet_name + "\n" + date)
    worksheet.set_footer('&L Generated on :&D &T &R &P of &N')
    worksheet.fit_to_pages(1, 0)
    worksheet.set_landscape() 
    writer.save()
    files.download(sheet_name + '.xlsx')

var_analysis()    
    